In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import matplotlib.pyplot as plt

In [3]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

path = 'drive/MyDrive/AgriLifeMicrohistologicalProject/GrassImages'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
#image_size = (1100,1800)
image_size = (600,600)

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train = train_datagen.flow_from_directory(
    path,
    target_size=image_size,
    batch_size=24,
    class_mode='sparse',
    subset='training'
)

valid = train_datagen.flow_from_directory(
    path,
    target_size=image_size,
    batch_size=4,
    class_mode='sparse',
    subset='validation'
)

#test = train_datagen.flow_from_directory(
 #   path,
  #  target_size=image_size,
   # batch_size=batch_size,
   # class_mode='sparse',
   # shuffle=False,
    #subset='validation'
#)

def plotimg(images_arr):
  fig,axes = plt.subplots(1,24)
  axes = axes.flatten()
  for img,ax in zip(images_arr,axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()

Found 24 images belonging to 5 classes.
Found 4 images belonging to 5 classes.


In [18]:
X_train, y_train = train[0]
X_val, y_val = valid[0]
#plotimg(X_train)
#print(y_train)

#0 - nale
#1 - qufu
#2 - erci
#3 - bubble
#4 - qufu stem

In [19]:
X_train = X_train / 255.0
X_val = X_val / 255.0

print(X_train.shape)

(24, 600, 600, 3)


In [7]:
from keras import datasets, layers, models

cnn = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(600, 600, 3)), #-> conv layers
    #layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    #layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'), #fully connected layers
    #layers.BatchNormalization(),
    layers.Dense(32, activation='relu'),
    #layers.BatchNormalization(),
    layers.Dense(10, activation='softmax'),
    #layers.BatchNormalization(),
])

cnn.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

cnn.fit(X_train, y_train, epochs=35)

Epoch 1/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 28s 28s/step - accuracy: 0.0833 - loss: 2.3027
Epoch 2/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 32s 32s/step - accuracy: 0.2083 - loss: 2.2311
Epoch 3/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.2083 - loss: 2.0710
Epoch 4/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - accuracy: 0.2083 - loss: 1.9126
Epoch 5/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 21s 21s/step - accuracy: 0.2083 - loss: 1.8864
Epoch 6/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.2083 - loss: 1.8112
Epoch 7/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - accuracy: 0.3333 - loss: 1.7917
Epoch 8/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 21s 21s/step - accuracy: 0.2083 - loss: 1.7574
Epoch 9/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 21s 21s/step - accuracy: 0.2083 - loss: 1.7001
Epoch 10/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - accuracy: 0.2083 - loss: 1.6820
Epoch 11/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 21s 21s/step - accuracy: 0.2083 - loss: 1.6515
Epoch 12/35
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - accuracy: 0.2083 - loss: 1.6074
E

In [14]:
X_val.shape

(4, 600, 600, 3)

In [20]:
cnn.evaluate(X_val, y_val)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5000 - loss: 2.4917


[2.491722822189331, 0.5]

In [21]:
y_predicted = cnn.predict(X_val)
y_predicted_labels = [np.argmax(i) for i in y_predicted]
print(y_predicted_labels[:4])
print(y_val[:4])

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 946ms/step
[1, 0, 2, 0]
[1. 0. 3. 2.]
